# Práctica 05: Algoritmo Genético

## Juego de la vida con Algoritmo Genético 

Reglas:
- Nacimientos: Una célula muerta con exactamente tres vecinos vivos se convierte en una célula viva.
- Muerte uno: Una célula viva con uno o menos vecinos vivos muere.
- Muerte dos: Una célula viva con más de tres vecinos vivos muere, a menos que se  cumpla la condición especial.
- Condición Especial de Supervivencia: Si la población de cromosomas alcanza o  supera un $n$ especifica, las células vivas pueden soportar hasta cuatro  vecinos vivos sin morir.
- Supervivencia Normal: Si no se cumple la condición especial, una célula viva con dos o tres vecinos vivos sobrevive.
- Muerte tres: Si en $n$ generaciones no se alcanza el objetivo el juego termina.

<center><h1>INICIO DEL CÓDIGO</h1></center>

In [2]:
import random

import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt
import numpy as np 

# Constantes
TABLERO = 10
POBLACION_INICIAL = 850
GENERACIONES = 160
# porcentaje de las células vivas del tablero esperadas
OBJETIVO_CELULAS_VIVAS = 0.09 * (TABLERO * TABLERO)  
C = 1  # Velocidad de la luz (celda por generación)

# Constantes para los límites del tablero
LIMITE_SUPERIOR = 0
LIMITE_INFERIOR = TABLERO - 1
LIMITE_IZQUIERDO = 0
LIMITE_DERECHO = TABLERO - 1

# Función para inicializar la población
def inicializar_poblacion(tamano):
    poblacion = []
    for _ in range(tamano):
        cromosoma = [random.randint(0, 1) for _ in range(TABLERO**2)]
        poblacion.append(cromosoma)
    return poblacion

# Función fitness basada en la velocidad de las astronaves
def funcion_fitness(cromosoma):
    x, y, n, toca_borde = calcular_desplazamiento(cromosoma)
    v = max(abs(x), abs(y)) / n * C
    # Penalizar la aptitud si la nave espacial toca un borde
    if toca_borde:
        v *= 0.5  # Ajusta esta penalización según sea necesario
    return v    

# Función para calcular el desplazamiento de las naves espaciales
def calcular_desplazamiento(cromosoma):
    x_actual, y_actual = obtener_posicion_nave(cromosoma)
    desplazamiento_x = 0
    desplazamiento_y = 0
    toca_borde = False
    
    # Verificar si la nave espacial ha tocado algún borde    
    if x_actual <= LIMITE_SUPERIOR:
        desplazamiento_x = 1  # Mover hacia abajo
    elif x_actual >= LIMITE_INFERIOR:
        desplazamiento_x = -1  # Mover hacia arriba
    if y_actual <= LIMITE_IZQUIERDO:
        desplazamiento_y = 1  # Mover hacia la derecha
    elif y_actual >= LIMITE_DERECHO:
        desplazamiento_y = -1  # Mover hacia la izquierda

    generaciones = 1
    return desplazamiento_x, desplazamiento_y, generaciones, toca_borde


# Función para obtener la posición actual de la nave espacial
def obtener_posicion_nave(cromosoma):
    # Buscar en el cromosoma la primera célula viva que podría ser parte de una nave espacial
    for i in range(TABLERO):
        for j in range(TABLERO):
            # Suponiendo que 1 representa una célula viva
            if cromosoma[i * TABLERO + j] == 1:
                # Retorna la posición (i, j) de la célula viva encontrada
                return i, j
    # Si no se encuentra ninguna célula viva, retorna -1, -1
    return -1, -1

# Función para aplicar las reglas del Juego de la Vida y evaluar el tablero
def evaluar_tablero(cromosoma, generacion_actual, max_generaciones=GENERACIONES, objetivo_celulas_vivas=OBJETIVO_CELULAS_VIVAS):
    tablero = [cromosoma[i:i+TABLERO] for i in range(0, len(cromosoma), TABLERO)]
    nuevo_tablero = [[0]*TABLERO for _ in range(TABLERO)]
    celulas_vivas = 0  # Contador para el número de células vivas
    
    for i in range(TABLERO):
        for j in range(TABLERO):
            vecinos_vivos = contar_vecinos_vivos(tablero, i, j)
            if tablero[i][j] == 1:  # Si la célula está viva
                if vecinos_vivos < 2 or vecinos_vivos > 3:
                    nuevo_tablero[i][j] = 0  # Muerte por aislamiento
                else:
                    nuevo_tablero[i][j] = 1  # Supervivencia
                    celulas_vivas += 1
            else:  # Si la célula está muerta
                if vecinos_vivos == 3:
                    nuevo_tablero[i][j] = 1  # Nacimiento de una nueva célula
                    celulas_vivas += 1
    
    # Convertir el nuevo tablero a un cromosoma para la siguiente generación
    nuevo_cromosoma = [celula for fila in nuevo_tablero for celula in fila]
    
    # Muerte tres: clausula del juego si esque no se alcanza el objetivo en las generaciones
    if generacion_actual >= max_generaciones and celulas_vivas < objetivo_celulas_vivas:
        return None  # Esto indicaría el fin del juego
    
    return nuevo_cromosoma

# Función para contar los vecinos vivos de una célula
def contar_vecinos_vivos(tablero, fila, columna):
    vecinos_vivos = 0
    for i in range(max(0, fila-1), min(fila+2, TABLERO)):
        for j in range(max(0, columna-1), min(columna+2, TABLERO)):
            if (i, j) != (fila, columna) and tablero[i][j] == 1:
                vecinos_vivos += 1
    return vecinos_vivos

# Función de selección por torneo
def seleccion_por_torneo(poblacion, aptitudes):
    seleccionados = []
    tamano_de_poblacion = len(poblacion)
    if tamano_de_poblacion < 2:
       return seleccionados
    
    for _ in range(len(poblacion)):
       if tamano_de_poblacion >= 2:
           competidores = random.sample(list(enumerate(aptitudes)), 2)
           ganador = max(competidores, key=lambda item: item[1])
           seleccionados.append(poblacion[ganador[0]])
           tamano_de_poblacion -=1
       else:
           break
    return seleccionados

# Función de reproducción con cruce de un punto
def reproduccion(seleccionados):
    hijos = []
    while len(seleccionados) > 1:  # Asegurarse de que hay al menos dos padres para el cruce
        padre1 = seleccionados.pop(random.randint(0, len(seleccionados) - 1))
        padre2 = seleccionados.pop(random.randint(0, len(seleccionados) - 1))
        
        # Elegir un punto de cruce al azar
        punto_cruce = random.randint(1, len(padre1) - 1)
        
        # Crear dos hijos a partir de los padres
        hijo1 = padre1[:punto_cruce] + padre2[punto_cruce:]
        hijo2 = padre2[:punto_cruce] + padre1[punto_cruce:]
        
        # Añadir los hijos a la nueva población
        hijos.append(hijo1)
        hijos.append(hijo2)
    
    # Si queda un padre sin pareja, se añade directamente a la población de hijos
    if seleccionados:
        hijos.append(seleccionados[0])
    
    return hijos


# Función de mutación
def mutacion(hijos):
    mutados = []
    for hijo in hijos:
        if random.random() < 0.1:  # Probabilidad de mutación del 10%
            indice = random.randint(0, len(hijo) - 1)
            hijo[indice] = 1 if hijo[indice] == 0 else 0
        mutados.append(hijo)
    return mutados

# Función de reemplazo
def reemplazo(poblacion, mutados, aptitudes):
    # Reemplaza los peores ajustados con los nuevos mutados
    ordenados = sorted(zip(poblacion, aptitudes), key=lambda item: item[1], reverse=True)
    sobrevivientes = ordenados[:len(poblacion) - len(mutados)]
    nueva_poblacion = [individuo for individuo, _ in sobrevivientes] + mutados
    return nueva_poblacion


# Función para imprimir el tablero
def imprimir_tablero(cromosoma):
    tablero = [cromosoma[i:i+TABLERO] for i in range(0, len(cromosoma), TABLERO)]
    for fila in tablero:
        print(' '.join(['X' if celula == 1 else '-' for celula in fila]))
    print('\n')

def contar_celulas_vivas(poblacion):
    # Contar el número de células vivas en la población
    return sum(1 for cromosoma in poblacion for celula in cromosoma if celula == 1)

# Método principal que ejecuta el algoritmo genético
def main():
    poblacion = inicializar_poblacion(POBLACION_INICIAL)
    poblacion_actual = len(poblacion)  # Asumiendo que poblacion actual es la longitud de la poblacion
    # Creamos laa figura con sus ejes
    figura, ejes = plt.subplots()
    
    # Definir el punto de control en el 70% de las generaciones
    punto_control = int(GENERACIONES * 0.7)

    for generacion_actual in range(GENERACIONES):
        # Aplicar las reglas del juego y evaluar el tablero
        nueva_poblacion = []
        for cromosoma in poblacion:
            nuevo_cromosoma = evaluar_tablero(cromosoma, poblacion_actual, generacion_actual)
            if nuevo_cromosoma is not None:
                nueva_poblacion.append(nuevo_cromosoma)
                
        poblacion = nueva_poblacion

        # Verificar si se ha alcanzado el punto de control
        if generacion_actual >= punto_control:
            # Contar células vivas y verificar si se cumple el objetivo
            celulas_vivas = contar_celulas_vivas(poblacion)
            if celulas_vivas < OBJETIVO_CELULAS_VIVAS:
                print(f"La simulación ha terminado en la generación {generacion_actual + 1} porque el número de células vivas ({celulas_vivas}) es menor que el objetivo ({OBJETIVO_CELULAS_VIVAS}).")
                break
        
        if not poblacion:  # Si la población estaa vacia deberemos terminar el juego
            # Este mensaje se obtiene si es que ninguna celula fue apta para continuar
            #  por lo tanto tenemos una población vacia
            print("El juego ha terminado debido a que ninguna celula fue apta para una nueva generación")
            break
                
                
        # Creamos la interfaz gráfica
        tablero = np.array(poblacion[0]).reshape((TABLERO, TABLERO))
        ejes.clear()
        ejes.imshow(tablero, cmap='Greys', interpolation='nearest')
        ejes.set_title(f'Generación: {generacion_actual + 1}')
        plt.pause(0.1)

        # Calcular la aptitud de cada cromosoma basada en las astronaves
        aptitudes = [funcion_fitness(cromosoma) for cromosoma in poblacion]
        
        # Asegurarse de que hay suficientes individuos para la selección por torneo
        if len(poblacion) < 2:
            print(f"No hay suficientes individuos para continuar la selección por torneo en la generación {generacion_actual + 1}.")
            break

        # Selección por torneo
        seleccionados = seleccion_por_torneo(poblacion, aptitudes)
        
        # Reproducción sexual
        hijos = reproduccion(seleccionados)
        
        # Mutación
        mutados = mutacion(hijos)
        
        # Reemplazo
        poblacion = reemplazo(poblacion, mutados, aptitudes)

    # Imprimir la generación final
    print(f"El juego alcanzo la generacion: {generacion_actual + 1}")

    # Texto de finalizeción 
    plt.text(0.5, 0.1, 'Proceso Terminado - Puede cerrar esta ventana', 
        horizontalalignment='center', 
        verticalalignment='center', 
        transform=plt.gcf().transFigure,
        fontsize=15)    
    # Se imprime la animación
    plt.show()

    
if __name__ == "__main__":
    main()

El juego alcanzo la generacion: 160
